# Sample Jupyter Notebook in Clojure for JQuants

Start by installing clojupyter. using:

```bash
# https://cljdoc.org/d/nubank/clojupyter/0.2.2/doc/readme#installation
git clone https://github.com/clojupyter/clojupyter    
make && make install   
```
Then you can start the jupyter console as usual:
``` bash
jupyter-console
```

In [1]:
(require '[clojupyter.misc.helper :as helper])
; (helper/add-dependencies '[metasoarous/oz "2.0.0-alpha5"])
(helper/add-dependencies '[metasoarous/oz "1.5.6"])
(helper/add-dependencies '[net.clojars.hellonico/jquants-api-jvm "0.2.11"])

{[org.xerial/sqlite-jdbc "3.39.3.0"] nil, [com.github.seancorfield/next.jdbc "1.3.828"] #{[org.clojure/java.data "1.0.95"] [camel-snake-kebab "0.4.3"]}, [org.clojure/java.data "1.0.95"] nil, [com.fasterxml.jackson.dataformat/jackson-dataformat-cbor "2.13.3" :exclusions [[com.fasterxml.jackson.core/jackson-databind]]] nil, [org.clojure/tools.logging "1.2.4"] nil, [com.github.seancorfield/honeysql "2.3.928"] nil, [org.clojure/clojure "1.11.1"] #{[org.clojure/core.specs.alpha "0.2.62"] [org.clojure/spec.alpha "0.3.218"]}, [tigris "0.1.2"] nil, [org.slf4j/slf4j-api "2.0.2"] nil, [net.clojars.hellonico/jquants-api-jvm "0.2.11"] #{[org.xerial/sqlite-jdbc "3.39.3.0"] [com.github.seancorfield/next.jdbc "1.3.828"] [org.clojure/tools.logging "1.2.4"] [com.github.seancorfield/honeysql "2.3.928"] [org.clojure/clojure "1.11.1"] [org.slf4j/slf4j-api "2.0.2"] [ch.qos.logback/logback-classic "1.4.1"] [cheshire "5.11.0"] [http-kit "2.3.0"] [ch.qos.logback/logback-core "1.4.1"]}, [ch.qos.logback/logback

In [2]:
(require '[hellonico.jquants-api :as api])
(require '[oz.notebook.clojupyter :as oz])

nil

# Draw a simple graph with oz in Jupyter

In [3]:
(defn datapt [[s n]] {:Index s, :Value n})
(defn graph  [vs]    {:data {:values vs}, 
                      :mark :line
                      :encoding {:x {:field :Index}
                                 :y {:field :Value}}})
(def graph-it (comp graph (partial map datapt) (partial apply map vector)));

In [4]:
(defn raw-data [] (map #(+ % (rand-int 25)) (range)))

#'user/raw-data

In [5]:
(->> [(range) (take 150 (raw-data))]
     graph-it
     oz/view!)

# Get the quotes from jquants and draw them using oz

In [6]:
(def data (:daily_quotes (api/daily-fuzzy {:CompanyNameEnglish "M3"})))
; (def raw (api/daily {:code 2413 :from 20220301 :to 20220328}))

(defn average [coll]
  (/ (reduce + coll)
     (count coll)))

(defn ma [period coll]
  (lazy-cat (repeat (dec period) nil)
            (map average (partition period 1  coll))))
(def ma-5 (partial ma 5))

(def _data (map #(assoc %1 :ma %2) data (ma-5 (map :High data))))

(def data-plot
  {:data {:values _data}
   :encoding {:x {:field "Date" :type "ordinal"}
              :y {:field "ma" :type "quantitative"}
              :color {:field "Code" :type "nominal"}}
   :mark "line"}
  )

(def viz
  [:div
   [:h1 "5 days Moving average"]
   [:p "Entity code 24130"]
   [:p "from 20220301 to 20220328"]
   [:vega-lite data-plot]
   ;[:h2 "If ever, oh ever a viz there was, the vizard of oz is one because, because, because..."]
   [:p (str (java.util.Date.))]
   ])

{:CompanyNameEnglish M3}


{"daily_quotes": [{"Code": "24130", "Close": 2978.0, "Date": "20170104", "AdjustmentHigh": 1515.0, "Volume": 1297500.0, "TurnoverValue": 3868135700.0, "AdjustmentClose": 1489.0, "AdjustmentLow": 1478.5, "Low": 2957.0, "High": 3030.0, "Open": 3000.0, "AdjustmentOpen": 1500.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2595000.0}, {"Code": "24130", "Close": 3050.0, "Date": "20170105", "AdjustmentHigh": 1525.0, "Volume": 1310400.0, "TurnoverValue": 3965351500.0, "AdjustmentClose": 1525.0, "AdjustmentLow": 1477.5, "Low": 2955.0, "High": 3050.0, "Open": 2971.0, "AdjustmentOpen": 1485.5, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2620800.0}, {"Code": "24130", "Close": 3100.0, "Date": "20170106", "AdjustmentHigh": 1560.0, "Volume": 1557100.0, "TurnoverValue": 4809893000.0, "AdjustmentClose": 1550.0, "AdjustmentLow": 1527.5, "Low": 3055.0, "High": 3120.0, "Open": 3085.0, "AdjustmentOpen": 1542.5, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3114200.0}, {"Code": "24130", "Close": 3150.0, "D

tHigh": 1458.0, "Volume": 1273900.0, "TurnoverValue": 3679566000.0, "AdjustmentClose": 1440.0, "AdjustmentLow": 1435.5, "Low": 2871.0, "High": 2916.0, "Open": 2915.0, "AdjustmentOpen": 1457.5, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2547800.0}, {"Code": "24130", "Close": 2838.0, "Date": "20170216", "AdjustmentHigh": 1454.5, "Volume": 1593800.0, "TurnoverValue": 4549203400.0, "AdjustmentClose": 1419.0, "AdjustmentLow": 1415.0, "Low": 2830.0, "High": 2909.0, "Open": 2902.0, "AdjustmentOpen": 1451.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3187600.0}, {"Code": "24130", "Close": 2875.0, "Date": "20170217", "AdjustmentHigh": 1440.0, "Volume": 1905900.0, "TurnoverValue": 5438015600.0, "AdjustmentClose": 1437.5, "AdjustmentLow": 1410.5, "Low": 2821.0, "High": 2880.0, "Open": 2826.0, "AdjustmentOpen": 1413.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3811800.0}, {"Code": "24130", "Close": 2845.0, "Date": "20170220", "AdjustmentHigh": 1448.0, "Volume": 1416500.0, "TurnoverValue": 4

5, "AdjustmentLow": 1404.0, "Low": 2808.0, "High": 2853.0, "Open": 2829.0, "AdjustmentOpen": 1414.5, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3537800.0}, {"Code": "24130", "Close": 2792.0, "Date": "20170330", "AdjustmentHigh": 1419.5, "Volume": 998100.0, "TurnoverValue": 2798146400.0, "AdjustmentClose": 1396.0, "AdjustmentLow": 1391.0, "Low": 2782.0, "High": 2839.0, "Open": 2835.0, "AdjustmentOpen": 1417.5, "AdjustmentFactor": 1.0, "AdjustmentVolume": 1996200.0}, {"Code": "24130", "Close": 2763.0, "Date": "20170331", "AdjustmentHigh": 1406.0, "Volume": 1039600.0, "TurnoverValue": 2898845700.0, "AdjustmentClose": 1381.5, "AdjustmentLow": 1381.5, "Low": 2763.0, "High": 2812.0, "Open": 2798.0, "AdjustmentOpen": 1399.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2079200.0}, {"Code": "24130", "Close": 2807.0, "Date": "20170403", "AdjustmentHigh": 1409.5, "Volume": 1021600.0, "TurnoverValue": 2858387500.0, "AdjustmentClose": 1403.5, "AdjustmentLow": 1381.0, "Low": 2762.0, "High": 2819.

ntOpen": 1492.5, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2145200.0}, {"Code": "24130", "Close": 3025.0, "Date": "20170515", "AdjustmentHigh": 1515.0, "Volume": 1255600.0, "TurnoverValue": 3781025900.0, "AdjustmentClose": 1512.5, "AdjustmentLow": 1486.5, "Low": 2973.0, "High": 3030.0, "Open": 2985.0, "AdjustmentOpen": 1492.5, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2511200.0}, {"Code": "24130", "Close": 3015.0, "Date": "20170516", "AdjustmentHigh": 1532.5, "Volume": 1252600.0, "TurnoverValue": 3795385000.0, "AdjustmentClose": 1507.5, "AdjustmentLow": 1502.5, "Low": 3005.0, "High": 3065.0, "Open": 3040.0, "AdjustmentOpen": 1520.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2505200.0}, {"Code": "24130", "Close": 3050.0, "Date": "20170517", "AdjustmentHigh": 1532.5, "Volume": 1109300.0, "TurnoverValue": 3374618500.0, "AdjustmentClose": 1525.0, "AdjustmentLow": 1500.0, "Low": 3000.0, "High": 3065.0, "Open": 3000.0, "AdjustmentOpen": 1500.0, "AdjustmentFactor": 1.0, "AdjustmentVo

...

tHigh": 3829.0, "Volume": 2755500.0, "TurnoverValue": 10310646400.0, "AdjustmentClose": 3703.0, "AdjustmentLow": 3701.0, "Low": 3701.0, "High": 3829.0, "Open": 3760.0, "AdjustmentOpen": 3760.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2755500.0}, {"Code": "24130", "Close": 3599.0, "Date": "20220527", "AdjustmentHigh": 3775.0, "Volume": 3569100.0, "TurnoverValue": 12982938300.0, "AdjustmentClose": 3599.0, "AdjustmentLow": 3593.0, "Low": 3593.0, "High": 3775.0, "Open": 3773.0, "AdjustmentOpen": 3773.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3569100.0}, {"Code": "24130", "Close": 3737.0, "Date": "20220530", "AdjustmentHigh": 3743.0, "Volume": 6121600.0, "TurnoverValue": 22740728900.0, "AdjustmentClose": 3737.0, "AdjustmentLow": 3642.0, "Low": 3642.0, "High": 3743.0, "Open": 3690.0, "AdjustmentOpen": 3690.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 6121600.0}, {"Code": "24130", "Close": 3737.0, "Date": "20220531", "AdjustmentHigh": 3782.0, "Volume": 3958000.0, "TurnoverValue"

00.0, "AdjustmentClose": 4264.0, "AdjustmentLow": 4039.0, "Low": 4039.0, "High": 4319.0, "Open": 4048.0, "AdjustmentOpen": 4048.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 4749900.0}, {"Code": "24130", "Close": 4246.0, "Date": "20220707", "AdjustmentHigh": 4298.0, "Volume": 3357200.0, "TurnoverValue": 14249257100.0, "AdjustmentClose": 4246.0, "AdjustmentLow": 4167.0, "Low": 4167.0, "High": 4298.0, "Open": 4247.0, "AdjustmentOpen": 4247.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3357200.0}, {"Code": "24130", "Close": 4136.0, "Date": "20220708", "AdjustmentHigh": 4268.0, "Volume": 4438500.0, "TurnoverValue": 18491424000.0, "AdjustmentClose": 4136.0, "AdjustmentLow": 4109.0, "Low": 4109.0, "High": 4268.0, "Open": 4267.0, "AdjustmentOpen": 4267.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 4438500.0}, {"Code": "24130", "Close": 4054.0, "Date": "20220711", "AdjustmentHigh": 4260.0, "Volume": 3169300.0, "TurnoverValue": 12975221800.0, "AdjustmentClose": 4054.0, "AdjustmentLow": 40

.0, "Low": 4888.0, "High": 5059.0, "Open": 4940.0, "AdjustmentOpen": 4940.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3653900.0}, {"Code": "24130", "Close": 4872.0, "Date": "20220819", "AdjustmentHigh": 5101.0, "Volume": 3779900.0, "TurnoverValue": 18708813700.0, "AdjustmentClose": 4872.0, "AdjustmentLow": 4867.0, "Low": 4867.0, "High": 5101.0, "Open": 4999.0, "AdjustmentOpen": 4999.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3779900.0}, {"Code": "24130", "Close": 4843.0, "Date": "20220822", "AdjustmentHigh": 4918.0, "Volume": 2605600.0, "TurnoverValue": 12663517000.0, "AdjustmentClose": 4843.0, "AdjustmentLow": 4777.0, "Low": 4777.0, "High": 4918.0, "Open": 4790.0, "AdjustmentOpen": 4790.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2605600.0}, {"Code": "24130", "Close": 4701.0, "Date": "20220823", "AdjustmentHigh": 4791.0, "Volume": 3820200.0, "TurnoverValue": 17941100900.0, "AdjustmentClose": 4701.0, "AdjustmentLow": 4657.0, "Low": 4657.0, "High": 4791.0, "Open": 4774.0, "

tmentOpen": 4154.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 4369200.0}, {"Code": "24130", "Close": 4071.0, "Date": "20221003", "AdjustmentHigh": 4086.0, "Volume": 2931900.0, "TurnoverValue": 11754627500.0, "AdjustmentClose": 4071.0, "AdjustmentLow": 3879.0, "Low": 3879.0, "High": 4086.0, "Open": 4000.0, "AdjustmentOpen": 4000.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 2931900.0}, {"Code": "24130", "Close": 4285.0, "Date": "20221004", "AdjustmentHigh": 4295.0, "Volume": 3128200.0, "TurnoverValue": 13260379800.0, "AdjustmentClose": 4285.0, "AdjustmentLow": 4116.0, "Low": 4116.0, "High": 4295.0, "Open": 4125.0, "AdjustmentOpen": 4125.0, "AdjustmentFactor": 1.0, "AdjustmentVolume": 3128200.0}]}


#'user/viz

In [7]:
(oz/view! viz)

5 days Moving average Entity code 24130 from 20220301 to 20220328